In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.general.make_noise import beep_on_error, beep_repeat
beep_on_error()

In [ ]:
import pandas as pd
import numpy as np
from src.data_tools.filename_skim_tools import regex_select, get_files, make_df
from src.data_tools.bff_meta import preselection, band_cut, isin, identity, all_reg, jet_variations
from src.data_tools.bff_cuts import *

from glob import glob
import re

In [ ]:
def one_in(list_of_strings, test_string):
    for string_in in list_of_strings:
        if string_in in test_string: return True
    return False

In [ ]:
from src.data_tools.get_file_list import get_file_df
from src.data_tools.get_data import get_data

In [ ]:
output_dir = 'assets_nov_2022'

In [ ]:
era = 2016
file_df = get_file_df(path = "{}/data".format(output_dir))
file_df = file_df[file_df.era==era]
file_df.reset_index(inplace=True)

In [ ]:
from src.data_tools.apply_cuts_v2 import  process_sample

In [ ]:
first_iter = 0
for i, row in file_df.iterrows():
    print("-----")
    df = process_sample(row, era, verbose=True)
    if first_iter==0: 
        combined_df = df
        first_iter=1
    else: combined_df = pd.concat([combined_df, df])
        
np.unique(df.SR1_jet_nom_muon_corrected_pt_ele_pt)

# convert to bool
combined_df['TriggerWeight'] = combined_df['TriggerWeight'].astype(bool)
combined_df['type'] = combined_df['type'].astype(str)
combined_df['name'] = combined_df['name'].astype(str)
combined_df['category'] = combined_df['category'].astype(str)

import pyarrow.feather as feather
feather.write_feather(combined_df, '{}/data/combined_{}.feather'.format(output_dir,era))

combined_df.to_parquet('{}/data/combined_{}.feather'.format(output_dir, era))

In [ ]:
beep_repeat()

In [ ]:
beep_repeat()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
bck_df = combined_df[(combined_df.type=='bck')]
no_bff = bck_df[(bck_df.SR1_nom > 0 )]
post_bff = bck_df[(bck_df.SR1_nom ==1)]
from src.plotting_tools.Bins import make_bins
bins=make_bins()

In [ ]:
counts, _  = np.histogram(no_bff.DiLepMass,bins=bins.bin_edges, weights=no_bff.Weight)

In [ ]:
counts_cut, _  = np.histogram(post_bff.DiLepMass,bins=bins.bin_edges, weights=post_bff.Weight)

In [ ]:
plt.plot(bins.calc_bin_centers(), counts)
plt.plot(bins.calc_bin_centers(),counts_cut)


In [ ]:
plt.plot(bins.calc_bin_centers(),counts_cut/counts)
plt.ylim(top=1, bottom=0)

In [ ]:
#make counts for regions

columns = combined_df.iloc[0:1].filter(regex='[C|S]R.+nom.+ted_pt').columns

import re
cd = combined_df[combined_df.DiLepMass_jet_nom_muon_corrected_pt_ele_pt>120]
def get_sample_stats(name, cd):
    tdf = cd[cd.name==name]
    return {'name': name, **{re.findall('([C|S]R[0-9]+)', x)[0]:  tdf[tdf[x]==1].Weight.sum()  for x in columns}}

samples  = []
for name in combined_df.name.unique():
    print(name)
    samples.append(get_sample_stats(name, cd))
    
df = pd.DataFrame(samples)
df[['SR2','SR1','CR10', 'CR13', 'CR14', 'CR20','CR23','CR24']].round(2)

In [ ]:
print(df[['SR2','SR1','CR10', 'CR13', 'CR14', 'CR20','CR23','CR24']].round(1).to_latex())